In [6]:
!pip install IPython

In [7]:
import IPython

%pip install 'zenml[server]'
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2



#automatically restart kernel
IPython.Application.instance().kernel.do_shutdown(restart=True)

⠇ Installing integrations...


{'status': 'ok', 'restart': True}

In [15]:
NGROK_Token = '2rvbr3ajyM95H4t87f7zySl8y1R_5vLApwj5TimcPChYaoVf1'

In [16]:
from zenml.environment import Environment

if Environment.in_google_colab():
  !pip install pyngrok
  !ngrock authtoken (NGROK_Token)

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `ngrock authtoken (NGROK_Token)'


In [3]:
# remove any zenml files and initialise an empty repository
!rm -rf .zen
!zenml init

Initializing the ZenML global configuration version to 0.72.0
⠋ Initializing ZenML repository at /content.
Creating database tables
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializin

In [4]:
# dummy example to train a model
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

def train_data() -> None:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images),-1))
  X_train,X_test,y_train,y_test = train_test_split(data,digits.target,test_size=0.2,shuffle=False)

  model = SVC(gamma=0.001)
  model.fit(X_train,y_train)
  test_acc = model.score(X_test,y_test)
  print(f"Test accuracy: {test_acc}")

train_data()

Test accuracy: 0.9583333333333334


## Doing the above in Pipeline

1. Define the steps in a pipeline

In [8]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_test"]
]:
    ## Load the digits as numpy arrays
    digits = load_digits()
    data = digits.images.reshape((len(digits.images),-1))
    X_train,X_test,y_train,y_test = train_test_split(data,digits.target,test_size=0.3,shuffle=False)

    return X_train, X_test, y_train, y_test

@step
def svc_trainer(X_train:np.ndarray,
                y_train:np.ndarray) -> ClassifierMixin:

                model = SVC(gamma=0.001)
                model.fit(X_train,y_train)
                return model

@step
def evaluator(X_test : np.ndarray,
              y_test : np.ndarray,
              model = ClassifierMixin) ->float:

              test_acc = model.score(X_test,y_test)
              print(f"Test accuracy is {test_acc}")
              return test_acc

2. Connecting the steps in a pipeline

In [9]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train=X_train, y_train=y_train)
  evaluator(X_test,y_test,model)

3. Running the Pipeline

In [10]:
svc_digits_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Registered new pipeline: digits_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
Step svc_trainer has started.
Step svc_trainer has finished in 1.253s.
Step evaluator has started.
Test accuracy is 0.9703703703703703
Step evaluator has finished in 0.412s.
Pipeline run has finished in 11.264s.


4. Visit the dashboard

In [ ]:
NGROK_Token = '2rvbr3ajyM95H4t87f7zySl8y1R_5vLApwj5TimcPChYaoVf1'

In [21]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    ngrok.set_auth_token(NGROK_Token)

    public_url = ngrok.connect(port)
    print(f"Use this url instead: {public_url}")
    !zenml up --blocking --port {port}
  else:
    !zenml up --port {port}
start_zenml_dashboard()

INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok


Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-e682a235-3c2b-48ed-ae7f-bcd74d54e925


Opening tunnel named: http-8237-e682a235-3c2b-48ed-ae7f-bcd74d54e925


INFO:pyngrok.process.ngrok:t=2025-01-21T09:22:39+0000 lvl=info msg="no configuration paths supplied"


...

t=2025-01-21T09:22:40+0000 lvl=info msg=end pg=/api/tunnels id=fc910678ef96b542 status=201 dur=74.985198ms
The `zenml up` command is deprecated and will be removed in a future release. Please use the `zenml 
login --local` command instead.
Calling `zenml login --local`...
The local ZenML dashboard is about to deploy in a blocking process.
Deploying a local daemon ZenML server.
Not writing the global configuration to disk in a ZenML server environment.
Initializing the ZenML global configuration version to 0.72.0
Not writing the global configuration to disk in a ZenML server environment.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [39382]
INFO:     Waiting for application startup.
Not writing the global configuration to disk in a ZenML server environment.
Not writing the global configuration to disk in a ZenML server environment.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Pre

INFO:pyngrok.process.ngrok:t=2025-01-21T09:37:17+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2025-01-21T09:37:17+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:pyngrok.process.ngrok:t=2025-01-21T09:37:17+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


t=2025-01-21T09:37:17+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:     Finished server process [39382]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "uvloop/loop.pyx", line 1512, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1505, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1379, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 557, in uvloop.loop.Loop._run
  File "uvloop/loop.pyx", line 476, in uvloop.loop.Loop._on_idle
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.pyx", line 63, in uvloop.loop.Handle._run
  File "/usr/local/lib/python3.11/dist-p

In [ ]:
import os
from zenml.environment import Environment

# Replace this with an environment variable for security
NGROK_Token = '2rvbr3ajyM95H4t87f7zySl8y1R_5vLApwj5TimcPChYaoVf1'

def start_zenml_dashboard(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        # Authenticate with ngrok using the token
        ngrok.set_auth_token(NGROK_Token)

        # Start the ngrok tunnel
        public_url = ngrok.connect(port)
        print(f"Use this URL instead: {public_url.public_url}")

        # Start the ZenML dashboard on the specified port
        !zenml up --blocking --port {port}
    else:
        # Run ZenML dashboard locally
        !zenml up --port {port}

# Start the ZenML dashboard
start_zenml_dashboard()


INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok


Updating authtoken for default "config_path" of "ngrok_path": /root/.config/ngrok/ngrok


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-61da9c94-f8fd-4dd2-9a17-f223c51d0cde


Opening tunnel named: http-8237-61da9c94-f8fd-4dd2-9a17-f223c51d0cde


INFO:pyngrok.process.ngrok:t=2025-01-21T09:37:24+0000 lvl=info msg="no configuration paths supplied"


...

Use this URL instead: https://78ad-35-184-206-234.ngrok-free.app
t=2025-01-21T09:37:24+0000 lvl=info msg=start pg=/api/tunnels id=6ff0900cc0e74ad4


INFO:pyngrok.process.ngrok:t=2025-01-21T09:37:24+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-61da9c94-f8fd-4dd2-9a17-f223c51d0cde addr=http://localhost:8237 url=https://78ad-35-184-206-234.ngrok-free.app


t=2025-01-21T09:37:24+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-61da9c94-f8fd-4dd2-9a17-f223c51d0cde addr=http://localhost:8237 url=https://78ad-35-184-206-234.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2025-01-21T09:37:24+0000 lvl=info msg=end pg=/api/tunnels id=6ff0900cc0e74ad4 status=201 dur=43.647535ms


t=2025-01-21T09:37:24+0000 lvl=info msg=end pg=/api/tunnels id=6ff0900cc0e74ad4 status=201 dur=43.647535ms
The `zenml up` command is deprecated and will be removed in a future release. Please use the `zenml 
login --local` command instead.
Calling `zenml login --local`...
The local ZenML dashboard is about to deploy in a blocking process.
Deploying a local daemon ZenML server.
Not writing the global configuration to disk in a ZenML server environment.
Initializing the ZenML global configuration version to 0.72.0
Not writing the global configuration to disk in a ZenML server environment.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [42931]
INFO:     Waiting for application startup.
Not writing the global configuration to disk in a ZenML server environment.
Not writing the global configuration to disk in a ZenML server environment.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Pre